In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import entropy_estimators as ee

import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter

import models
from models import Renorm_Dynamic
from EI_calculation import approx_ei
use_cuda = torch.cuda.is_available()
#device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device =torch.device('cpu')

## Simple Markov Chain

In [ ]:
markov = torch.Tensor([[1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0], [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0], 
                       [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0], [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0],
                      [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0], [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0],
                      [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 0], [0,0,0,0,0,0,0,1]])
#markov = torch.Tensor([[1/3, 1/3, 1/3, 0], [1/3, 1/3, 1/3, 0], [1/3, 1/3, 1/3, 0],[0,0,0,1]])
def one_step(state):
    m = distributions.Categorical(state@markov)
    s_next = torch.nn.functional.one_hot(m.sample())*1.0
    return s_next
def generate_data(batch_size):
    distrs = distributions.Categorical(torch.ones(markov.size()[0])/markov.size()[0])
    state = distrs.sample([batch_size, 1]).squeeze(1)
    state_onehot = torch.nn.functional.one_hot(state, markov.size()[0])*1.0
    state_next = one_step(state_onehot)
    return state_onehot.to(device), state_next.to(device)
def test_model(batch_size,input_range,net,scale,nll):
    batch_size1 = batch_size*10
    state, state_next = generate_data(batch_size)
    predict, latent, latent_p = net(state) 
    predict = nn.functional.log_softmax(predict, dim=1)
    loss = nll(predict, torch.nonzero(state_next)[:,1])
    ssp = net.encoding(state_next)
    sigmas = torch.sqrt(torch.mean((ssp-latent_p)**2, 0))
    inv_sigma = torch.inverse(torch.diag(sigmas))
    ei = approx_ei(scale, scale, inv_sigma.data, lambda x:(net.dynamics(x.unsqueeze(0))+x.unsqueeze(0)), 1000, 100)
    one_ei = onehot_ei(input_range, lambda x: net(x))
    return ei, sigmas,loss.item(), one_ei
def onehot_ei(input_size, netfunc):
    values = torch.LongTensor(range(input_size))
    onehots = torch.nn.functional.one_hot(values)*1.0
    pyx,_,_ = netfunc(onehots)
    pyx = torch.softmax(pyx, dim=1)
    logpyx=torch.log(pyx)
    logpyx = torch.where(torch.isinf(logpyx), torch.zeros(onehots.size()), logpyx)
    entropy = pyx * logpyx
    sumz = torch.sum(pyx, 0).unsqueeze(0)
    logsumz = torch.log(sumz)
    logsumz = torch.where(torch.isinf(logsumz), torch.zeros(logsumz.size()), logsumz)
    logsumz = logsumz.repeat(input_size, 1)
    
    #print(logsumz)
    first_term = torch.sum(entropy)
    second_term = torch.sum(pyx * logsumz)
    
    final = (first_term - second_term)/input_size + np.log(input_size)
    final = final / np.log(2.0)
    return final, final*np.log(2.0)/np.log(input_size), pyx, first_term, second_term

In [ ]:
def RGB_to_Hex(rgb):

    RGB = list(rgb)
    color = '#'
    for i in RGB:
        num = int(i)
        color += str(hex(num))[-2:].replace('x', '0').upper()
    return color
    
def generate_colors(N=12,colormap='hsv'):

    step = max(int(255/N),1)
    cmap = plt.get_cmap(colormap)
    rgb_list = []
    hex_list = []
    for i in range(N):
        id = step*i # cmap(int)->(r,g,b,a) in 0~1
        id = 255 if id>255 else id
        rgba_color = cmap(id)
        rgb = [int(d*255) for d in rgba_color[:3]]
        rgb_list.append(tuple(rgb))
        hex_list.append(RGB_to_Hex(rgb))
    return rgb_list,hex_list
    
rgb_list,hex_list = generate_colors(6,'cool')
print(rgb_list)
print(hex_list)


### Learned Mapping

In [ ]:
hidden_units = 64
torch.manual_seed(50)
scale = 1
batch_size =100
nll = nn.NLLLoss()
net = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
#net = net.cuda() if use_cuda else net
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad==True], lr=1e-4)
random_samples = []
best_ei = 0
best_model = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                            hidden_units = hidden_units, normalized_state = False, device=device)
#best_model = best_model.cuda() if use_cuda else best_model
for t in range(500001):    
    state,state_next = generate_data(batch_size)
    predict, latent, latent_p = net(state)
    predict = nn.functional.log_softmax(predict, dim=1)
    loss = nll(predict, torch.nonzero(state_next)[:,1])
    
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
           
    if t % 500 ==0:
        ei, sigmas,_,one_hot_ei = test_model(batch_size, markov.size()[0], net,scale,nll)
        if ei[0]>best_ei:
            best_model.load_state_dict(net.state_dict())
            best_ei = ei[0]
        print('iter %s:' % t, 'loss = %.3f' % loss.item(), ', EI= %.3f' % ei[0],
             ', eff= %.3f' % ei[1], ', std = %.3f' % sigmas.mean().item(), ',onehot_EI = %.3f' % one_hot_ei[0], 
              ',onehot_ei = %.3f' % one_hot_ei[1])
        
        if t % 5000 == 0:
            ei, sigmas,_,_ = test_model(batch_size, markov.size()[0], best_model,scale,nll)
            print('best model: EI= %.3f' % ei[0],
                 ', eff= %.3f' % ei[1], ', std = %.3f' % sigmas.mean().item())
            xx = torch.sum(state * torch.linspace(0,state.size()[1]-1,state.size()[1], device=device),1)
            if latent.size()[1]==1:
                yy = latent
                if use_cuda:
                    xx=xx.cpu()
                    yy=yy.cpu()
                for i in range(yy.size()[1]):
                    plt.plot(xx.data, yy[:,i].data,'.')
                plt.xlabel('Learned Latent State')
                plt.ylabel('Real Latent State')
            else:
                lowrank=latent
                if latent.size()[1]>2:
                    lowrank = torch.pca_lowrank(latent, q=2)[0]
                plotx = xx.cpu() if use_cuda else xx
                ploty = lowrank.cpu() if use_cuda else lowrank
                plt.scatter(ploty.data[:, 0], ploty.data[:, 1], c = plotx,cmap=plt.cm.Spectral)
            plt.show()

In [ ]:
xx=torch.linspace(-1,1,100, device=device).unsqueeze(1)
#xx=xx.repeat(1, 2)
print(xx.size())
yy=net.dynamics(xx)+xx
#print(yy)
yy=yy.cpu() if use_cuda else yy
xx=xx.cpu() if use_cuda else xx
plt.plot(xx[:,0].data,yy[:,0].data,color='royalblue')
plt.xlabel('Latent Macro State at Current Time',fontsize=14)
plt.ylabel('Latent Macro State at Next Time',fontsize=14)
plt.savefig('Simple_Markov1.svg', dpi=600, format='svg')
plt.show()

In [ ]:
state=torch.eye(8)
if use_cuda:
    state=state.cpu()
latent=net.encoding(state).squeeze()
print(latent)
if use_cuda:
    latent=latent.cpu()
bools = latent>-0.25
xx=np.array(range(8))
plt.plot(xx[bools], latent[bools].data, 'o', markersize=4, label='Class 1',color='mediumblue')
plt.plot(xx[~bools],latent[~bools].data, 's', markersize=4, label='Class 2',color='orangered')
plt.xlabel('Encoded Micro State',fontsize=14)
plt.ylabel('Latent Macro State',fontsize=14)
plt.legend()
plt.savefig('Simple_Markov2.svg', dpi=600, format='svg')
plt.show()

### Multi-scale experiments

In [ ]:
hidden_units = 64
torch.manual_seed(50)
scale = 8
batch_size =100
nll = nn.NLLLoss()
net = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
#net = net.cuda() if use_cuda else net
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad==True], lr=1e-4)
random_samples = []
best_ei = 0
best_model = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                            hidden_units = hidden_units, normalized_state = False, device=device)
#best_model = best_model.cuda() if use_cuda else best_model
for t in range(50001):    
    state,state_next = generate_data(batch_size)
    predict, latent, latent_p = net(state)
    predict = nn.functional.log_softmax(predict, dim=1)
    loss = nll(predict, torch.nonzero(state_next)[:,1])
    
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
           
    if t % 500 ==0:
        ei, sigmas,_,one_hot_ei = test_model(batch_size, markov.size()[0], net,scale,nll)
        if ei[0]>best_ei:
            best_model.load_state_dict(net.state_dict())
            best_ei = ei[0]
        print('iter %s:' % t, 'loss = %.3f' % loss.item(), ', EI= %.3f' % ei[0],
             ', eff= %.3f' % ei[1], ', std = %.3f' % sigmas.mean().item(), ',onehot_EI = %.3f' % one_hot_ei[0], 
              ',onehot_ei = %.3f' % one_hot_ei[1])
        
        if t % 5000 == 0:
            ei, sigmas,_,_ = test_model(batch_size, markov.size()[0], best_model,scale,nll)
            print('best model: EI= %.3f' % ei[0],
                 ', eff= %.3f' % ei[1], ', std = %.3f' % sigmas.mean().item())

In [ ]:
L = 10
hidden_units = 64
sigma=1
#torch.manual_seed(2050)
experiments = 5
batch_size =100
epochs=50001
nll = nn.NLLLoss()

err_scale = []
ei_scale=[]
for scale in [8,7,6,5,4,3,2,1]:
    print('*********************',scale,'**************************')
    err_experiment=[]
    multi_err_experiment=[]
    ei_experiment=[]
    for experiment in range(experiments):
        net = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False,device=device)
        net_m= Renorm_Dynamic(sym_size = markov.size()[0], latent_size = markov.size()[0], effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False,device=device)
        #net = net.cuda() if use_cuda else net
        #net_m= net_m.cuda() if use_cuda else net
        optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad==True], lr=1e-4)
        optimizer_m = torch.optim.Adam([p for p in net_m.parameters() if p.requires_grad==True], lr=1e-4)
        best_ei = 0
        best_model = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale, effect_size = markov.size()[0], 
                                    hidden_units = hidden_units, normalized_state = False,device=device)
        #best_model = best_model.cuda() if use_cuda else best_model
        for t in range(epochs):    
            state,state_next = generate_data(batch_size)
            predict, latent, latent_p = net(state)
            predict_m, latent_m, latent_pm = net_m(state)
            predict = nn.functional.log_softmax(predict, dim=1)
            predict_m = nn.functional.log_softmax(predict_m, dim=1)
            loss = nll(predict, torch.nonzero(state_next)[:,1])
            loss_m = nll(predict_m, torch.nonzero(state_next)[:,1])

            optimizer.zero_grad()
            optimizer_m.zero_grad()
            loss.backward(retain_graph=True)
            loss_m.backward(retain_graph=True)
            optimizer.step()
            optimizer_m.step()

            if t % 5000 ==0:
                ei, sigmas,loss,one_ei = test_model(batch_size, markov.size()[0], net,scale, nll)
                ei_m, sigmas_m,loss_m,one_ei_m = test_model(batch_size, markov.size()[0], net_m, markov.size()[0], nll)
                if ei[0]>best_ei:
                    best_model.load_state_dict(net.state_dict())
                    best_ei=ei[0]
                print('iter %s:' % t, 'loss = %.3f' % loss, ', EI-EIm= %.3f' % (ei[0]-ei_m[0]),
                     ', eff= %.3f' % ei[1], ', std = %.3f' % sigmas.mean(), 
                      ',onehot_EI = %.3f' % one_hot_ei[0], ',onehot_ei = %.3f' % one_hot_ei[1])
        #test
        ei, sigmas,loss,_ = test_model(batch_size*10, markov.size()[0], net, scale, nll) 
        ei_m, sigmas_m,loss_m,_ = test_model(batch_size*10, markov.size()[0], net, scale, nll)
        err_experiment.append(loss)
        
        #mutual information
        ei_experiment.append(ei[0]-ei_m[0])
        
    err_scale.append(err_experiment)
    ei_scale.append(ei_experiment)

#### Casual Emergence

In [ ]:
rgb_list,hex_list = generate_colors(2,'cool')
print(rgb_list)
print(hex_list)
#Spring data
ces=ei_scale

scales=torch.Tensor([8,7,6,5,4,3,2,1])
means=[]
stds=[]

for ce in ces:
    m=np.mean(ce)
    std=np.std(ce)
    means.append(m)
    stds.append(std)

#plt.figure(figsize=(3.92*2,2.66*2), dpi = 80)
plt.plot(scales, means,'o')
plt.errorbar(scales, means, stds,color=hex_list[0])
plt.bar(scales, means, width=0.3, facecolor=hex_list[1], edgecolor='white')
plt.xlabel('Scale ($q$)',fontsize=13)
plt.ylabel('Casual Emergence (dCE)',fontsize=13)
plt.title('Relationship between dCE and Scale(q)',fontsize=14) 
plt.axhline(0, color='black', linestyle='--')
plt.savefig('Simple_casual_emergence.svg', dpi=600, format='svg')
plt.show()

### Mutual Information

In [ ]:
import entropy_estimators as ee

In [ ]:
hidden_units = 64
torch.manual_seed(50)
batch_size =100
nll = nn.NLLLoss()
scale = [6,5,4,3]
net_1 = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale[0], effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
net_2 = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale[1], effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
net_3 = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale[2], effect_size = markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
net_4 = Renorm_Dynamic(sym_size = markov.size()[0], latent_size = scale[3], effect_size =markov.size()[0], 
                     hidden_units = hidden_units, normalized_state = False, device=device)
#net = net.cuda() if use_cuda else net
optimizer_1 = torch.optim.Adam([p for p in net_1.parameters() if p.requires_grad==True], lr=1e-4)
optimizer_2 = torch.optim.Adam([p for p in net_2.parameters() if p.requires_grad==True], lr=1e-4)
optimizer_3 = torch.optim.Adam([p for p in net_3.parameters() if p.requires_grad==True], lr=1e-4)
optimizer_4 = torch.optim.Adam([p for p in net_4.parameters() if p.requires_grad==True], lr=1e-4)
random_samples = []

Isshat_1=[]
Isshat_2=[]
Isshat_3=[]
Isshat_4=[]
#Isshat_m=[]
I_xt_yt_1s=[]
I_xt_yt_2s=[]
I_xt_yt_3s=[]
I_xt_yt_4s=[]
#errs=[]
Ts=[]

for t in range(50001): 
    state,state_next=generate_data(batch_size)
    
    predict_1, latent_1, latent_p1 = net_1(state)
    predict_2, latent_2, latent_p2 = net_2(state)
    predict_3, latent_3, latent_p3 = net_3(state)
    predict_4, latent_4, latent_p4 = net_4(state)
    #predict = nn.functional.log_softmax(predict, dim=1)
    loss_1 = nll(predict_1, torch.nonzero(state_next)[:,1])
    loss_2 = nll(predict_2, torch.nonzero(state_next)[:,1])
    loss_3 = nll(predict_3, torch.nonzero(state_next)[:,1])
    loss_4 = nll(predict_4, torch.nonzero(state_next)[:,1])
    optimizer_1.zero_grad()
    optimizer_2.zero_grad()
    optimizer_3.zero_grad()
    optimizer_4.zero_grad()
    loss_1.backward(retain_graph=True)
    loss_2.backward(retain_graph=True)
    loss_3.backward(retain_graph=True)
    loss_4.backward(retain_graph=True)
    optimizer_1.step()
    optimizer_2.step()
    optimizer_3.step()
    optimizer_4.step()

    if t % 500 ==0:
        I_xt_xt1hat_1=ee.mi(state.cpu(),predict_1.cpu().detach(),k=9)
        I_xt_yt_1=ee.mi(state.cpu(),latent_1.cpu().detach(),k=9)

        I_xt_yt_2=ee.mi(state.cpu(),latent_2.cpu().detach(),k=9)
        I_xt_xt1hat_2=ee.mi(state.cpu(),predict_2.cpu().detach(),k=9)

        I_xt_xt1hat_3=ee.mi(state.cpu(),predict_3.cpu().detach(),k=9)
        I_xt_yt_3=ee.mi(state.cpu(),latent_3.cpu().detach(),k=9)

        I_xt_xt1hat_4=ee.mi(state.cpu(),predict_4.cpu().detach(),k=9)
        I_xt_yt_4=ee.mi(state.cpu(),latent_4.cpu().detach(),k=9)

        Isshat_1.append(I_xt_xt1hat_1)
        Isshat_2.append(I_xt_xt1hat_2)
        Isshat_3.append(I_xt_xt1hat_3)
        Isshat_4.append(I_xt_xt1hat_4)
        I_xt_yt_1s.append(I_xt_yt_1)
        I_xt_yt_2s.append(I_xt_yt_2)
        I_xt_yt_3s.append(I_xt_yt_3)
        I_xt_yt_4s.append(I_xt_yt_4)

        #errs.append(np.std([np.mean(I_yt_yt1),np.mean(I_xt_xt1hat),np.mean(I_xt_xt1)]))
        Ts.append(t)
        
        print(I_xt_xt1hat_4,I_xt_yt_1,I_xt_yt_2,I_xt_yt_3,I_xt_yt_4)

In [ ]:
def moving_average(interval, windowsize):
    window = np.ones(int(windowsize)) / float(windowsize)
    re = np.convolve(interval, window, 'same')
    return re

In [ ]:
Isshat_4_rol_mean = moving_average(Isshat_4,10)
Isshat_3_rol_mean = moving_average(Isshat_3,10)
Isshat_2_rol_mean = moving_average(Isshat_2,10)
Isshat_1_rol_mean = moving_average(Isshat_1,10)
I_xt_yt_1s_rol_mean = moving_average(I_xt_yt_1s,10)
I_xt_yt_2s_rol_mean = moving_average(I_xt_yt_2s,10)
I_xt_yt_3s_rol_mean = moving_average(I_xt_yt_3s,10)
I_xt_yt_4s_rol_mean = moving_average(I_xt_yt_4s,10)

In [ ]:
rgb_list,hex_list = generate_colors(12,'cubehelix')
print(rgb_list)
print(hex_list)
#plt.plot(Ts,Isshat_3,label='$I(x_t,\hat{x}_t)$',color='royalblue',linestyle='-.')
#plt.plot(Ts,Isshat_m,label='I(xt,xt_hat_m)')
#plt.plot(Ts,I_xt_yt_1s,label='$I(x_t,y_{t}^{q=2})$',color='orange',linestyle=':')
#plt.plot(Ts,I_xt_yt_2s,label='$I(x_t,y_{t}^{q=4})$',color='g',linestyle=':')
#plt.plot(Ts,I_xt_yt_3s,label='$I(x_t,y_{t}^{q=6})$',color='purple',linestyle=':')
#plt.plot(Ts,I_xt_yt_4s,label='$I(x_t,y_{t}^{q=8})$',color='dodgerblue',linestyle=':')
plt.plot(Ts[6:95],Isshat_1_rol_mean[6:95],label='$I(x_t,\hat{x}_t)$',color='forestgreen')
plt.plot(Ts[6:95],I_xt_yt_1s_rol_mean[6:95],label='$I(x_t,y_{t}^{q=6})$',color='purple')
plt.plot(Ts[6:95],I_xt_yt_2s_rol_mean[6:95],label='$I(x_t,y_{t}^{q=5})$',color='dodgerblue')
plt.plot(Ts[6:95],I_xt_yt_3s_rol_mean[6:95],label='$I(x_t,y_{t}^{q=4})$',color='crimson')
plt.plot(Ts[6:95],I_xt_yt_4s_rol_mean[6:95],label='$I(x_t,y_{t}^{q=3})$)',color='orange')
#plt.plot(Ts,I_xt_xt1hat_ms,label='I(xt,xthat)m')
#plt.plot(Ts,errs,label='error')
plt.legend(loc='best',fontsize=9)
plt.ylim(0.5,3.5)
plt.title('Verification for Theorem 6',fontsize=14) 
plt.xlabel('Iter',fontsize=13)
plt.ylabel('Mutual Information',fontsize=13)
plt.savefig('Markov_Mutual_Information_rm.svg', dpi=600, format='svg')
plt.show()